[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset

Let's first of all have a look at the data. You can download the file `labeledTrainData.tsv` on the [Kaggle website of the competition](https://www.kaggle.com/c/word2vec-nlp-tutorial/data), or on our [Google Drive](https://drive.google.com/file/d/1a1Lyn7ihikk3klAX26fgO3YsGdWHWoK5/view?usp=sharing)


In [15]:
# Import pandas, numpy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# Read dataset with extra params sep='\t', encoding="latin-1"
sentiment = pd.read_csv('/content/drive/My Drive/Data/DevC/labeledTrainData.tsv',sep='\t', encoding='latin-1')
sentiment.sample(5)

,id,sentiment,review
1379,9875_7,1,"Presenting Lily Mars (MGM, 1943) is a cute fil..."
3744,5657_9,1,I remember watching American Gothic when it fi...
23947,4592_9,1,Jackie Chan's Police Story is a landmark film ...
14792,11183_7,1,"\Pet Sematary\"" is an adaptation from the Step..."
5085,10773_1,0,I just watched this horrid thing on TV. Needle...


## 2. Preprocessing

In [17]:
# stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    # text = None # Your code here
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    # Your code here
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    # Your code here
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

print(preprocessor('Check done preprocessor'))

check done preprocessor 


In [19]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
# Your code here
porter = PorterStemmer()

# Split a text into list of words
def tokenizer(text):
    return text.split()

# Split a text into list of words and apply stemming technic
def tokenizer_porter(text):
    # return [] # Your code here
    return [porter.stem(word) for word in text.split()]

# Testing
print(tokenizer('Hi there, I am loving this, like with a lot of love'))
print(tokenizer_porter('Hi there, I am loving this, like with a lot of love'))

['Hi', 'there,', 'I', 'am', 'loving', 'this,', 'like', 'with', 'a', 'lot', 'of', 'love']
['Hi', 'there,', 'I', 'am', 'love', 'this,', 'like', 'with', 'a', 'lot', 'of', 'love']


In [25]:
# split the dataset in train and test
# Your code here
from sklearn.model_selection import train_test_split

X = sentiment['review']
y = sentiment['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=102)
print(preprocessor('Check done sentiment'))

check done sentiment 


## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [26]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=stop_words,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)

# A pipeline is what chains several steps together, once the initial exploration is done. 
# For example, some codes are meant to transform features — normalise numericals, or turn text into vectors, 
# or fill up missing data, they are transformers; other codes are meant to predict variables by fitting an algorithm,
# they are estimators. Pipeline chains all these together which can then be applied to training data
clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)


# tfidf = # Your code here

# clf = Pipeline(# Your code here)
# clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=<function preprocessor at 0x7f1d5bf16c80>,
                                 smooth_idf=True,
                                 stop_words=['i', 'me', 'my', 'myself', '...
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_porter at 0x7f1d5c0a70d0>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
         

## 4. Evaluate Model

In [28]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

predictions = clf.predict(X_test)
print('accuracy:',accuracy_score(y_test,predictions))
print('confusion matrix:\n',confusion_matrix(y_test,predictions))
print('classification report:\n',classification_report(y_test,predictions))


accuracy: 0.8854
confusion matrix:
 [[2164  339]
 [ 234 2263]]
classification report:
               precision    recall  f1-score   support

           0       0.90      0.86      0.88      2503
           1       0.87      0.91      0.89      2497

    accuracy                           0.89      5000
   macro avg       0.89      0.89      0.89      5000
weighted avg       0.89      0.89      0.89      5000



## 5. Export Model 

In [40]:
# Using pickle to export our trained model
import pickle
import os

pickle.dump(clf, open('week5Assignment.pkl', 'wb'))

In [41]:
twits = [
    "it so good",
    "it not bad",
    """As an avid reader, I am often disappointed by what is represented by the silver screen. I initially get excited to see one of my favorite series on the big screen, and eventually walk away disgusted. For whatever reason, most shows and movies that are based on epic book series absolutely butcher the original content.
So when I saw the preview for "Game of Thrones" on HBO, I was immediately skeptical. Oh, I was going to watch it, but I didn't have high expectations. I did hold a little hope because HBO is known for their high quality shows.
I was absolutely shocked to watch the Pilot and realize that it followed the book almost to perfection. The language that is used by the actors is very similar to the way that it was written in the book, and events were not skipped. It stayed very true to the story without adding to it or removing from it.
The acting is superb and I actually felt like I was looking into Winterfell for the first time. I feel like I've actually got faces to add to the characters that I've been reading about for a long time.
If you have not read the books, you may feel as though the blatant sexuality is an addition from HBO to add some spice to the series. This is not the case at all. The books contained all of the sexuality that is shown in the Pilot. I am very proud of HBO for producing an amazingly accurate product. I will be watching every week for sure.
If you enjoyed what you saw in the Pilot, know that there are many other plot twists and action filled evenings to come. "Game of Thrones" has a ton of potential to be one of the greatest series that has ever been produced. I would highly recommend it to fans of the series and newcomers alike.
""",
"""
It was a master piece. It was written to the perfection. It was mesmerizing. It was gripping. It was so shocking that if someone is binge watching this show he/she will need a time-off in between to get their head around things and accept some messed up, yet mind blowing development.
But yet, I cant hate it enough after final season. Its like you came to know that you were in love with the wrong one all along. It was like looking at a completely different person. It was like seeing your own dreams and expectations get destroyed. It was not a let down, it was a BETRAYAL!
"""

]

with open('week5Assignment.pkl', 'rb') as model:
    reload_model = pickle.load(model)
preds = reload_model.predict_proba(twits)



for i in range(len(twits)):
    print(f'{twits[i]} --> Negative, Positive = {preds[i]}')

it so good --> Negative, Positive = [0.11258079 0.88741921]
it not bad --> Negative, Positive = [9.99060595e-01 9.39404725e-04]
As an avid reader, I am often disappointed by what is represented by the silver screen. I initially get excited to see one of my favorite series on the big screen, and eventually walk away disgusted. For whatever reason, most shows and movies that are based on epic book series absolutely butcher the original content.
So when I saw the preview for "Game of Thrones" on HBO, I was immediately skeptical. Oh, I was going to watch it, but I didn't have high expectations. I did hold a little hope because HBO is known for their high quality shows.
I was absolutely shocked to watch the Pilot and realize that it followed the book almost to perfection. The language that is used by the actors is very similar to the way that it was written in the book, and events were not skipped. It stayed very true to the story without adding to it or removing from it.
The acting is supe

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
